## Defining dictionary, rules and running predictions

In [54]:
import os
import gc
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import inflect
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict, Counter
import feather
from num2words import num2words 
import re
import pickle

Read the train file, google dataset, test file with predicted classes

In [619]:
train = pd.read_csv('en_train.csv')
test = pd.read_csv('test_pred_class_new.csv')
test_old = pd.read_csv('test_pred_class.csv')
del test['Unnamed: 0.1']
del test_old['Unnamed: 0.1']
print(test_old.head())
test.head()

   Unnamed: 0  sentence_id  token_id     before  class
0           0            0         0    Another  PLAIN
1           1            0         1  religious  PLAIN
2           2            0         2     family  PLAIN
3           3            0         3         is  PLAIN
4           4            0         4         of  PLAIN


,Unnamed: 0,sentence_id,token_id,before,class
0,0,0,0,Last,PLAIN
1,1,0,1,modified,PLAIN
2,2,0,2,2016-03-31,DATE
3,3,0,3,.,PUNCT
4,4,1,0,There's,PLAIN


In [620]:
# d = train.groupby(['before', 'after']).size()
# d = d.reset_index().sort_values(0, ascending=False)
# d = d.loc[d['before'].drop_duplicates(keep='first').index]
# d = d.loc[d['before'] != d['after']]
# d = d.set_index('before')['after'].to_dict()
# test_kerem = pd.read_csv('test2_class.csv')
# test_kerem.head()
# # #Kerem
# test_kerem['before'] = test['before']
# test = test_kerem
# test['unnamed'] = ''
# test = test[['unnamed','sentence_id', 'token_id','before', 'class']]
# test.head()

In [621]:
d = defaultdict(list)
train_list = [(train.iloc[i,3],train.iloc[i,4]) for i in range(train.shape[0])]
for k,v in train_list:
    d[k].append(v)
    
train_dict = {}
for key in d:
    c = Counter(d[key]).most_common(1)[0][0]
    train_dict[key] = c


In [622]:
train_dict['V']

'V'

In [623]:
# extra_dict = pickle.load(open('extra_dict', 'rb'))
# extra_dict['-']

In [624]:
test_soln = pd.read_csv('en_solution.csv')
test_soln = test_soln.drop(['Usage'], axis = 1)
test_soln = test_soln.drop(['id'], axis = 1)
test_soln.head()
test_soln['before'] = test_old['before']
test_soln.head()


,after,before
0,Another,Another
1,religious,religious
2,family,family
3,is,is
4,of,of


In [625]:
d = defaultdict(list)
test_list = [(test_soln.iloc[i,1],test_soln.iloc[i,0]) for i in range(test_soln.shape[0])]
for k,v in test_list:
    d[k].append(v)
    
test_dict = {}
for key in d:
    c = Counter(d[key]).most_common(1)[0][0]
    test_dict[key] = c
del test_old, test_soln, test_list
test_dict['is']       


'is'

In [626]:
# extra_dict

In [627]:
test_dict['IV']

'i v'

In [628]:
# d1 = defaultdict(list)

# google_dataset = os.listdir('en_with_types/')
# for file in google_dataset:
#     data = pd.read_csv('en_with_types/' + file, sep = '\t', error_bad_lines= False, quoting=3)
#     print(file)
#     data = data[data.iloc[:,0] != 'PUNCT']
#     data = data[data.iloc[:,0] != 'ELECTRONIC']    
#     data = data[data.iloc[:,0] != '<eos>']   
#     data = data.drop(data.columns[0], axis=1)
#     #print(data.head())
    
#     data_list = [(data.iloc[i,0],data.iloc[i,1]) for i in range(data.shape[0])]
#     for k,v in data_list:
#         #print(data_list)[1:10]
#         #print(k)
#         #print(v)
#         d1[k].append(v)
#         #print(d1[k])
        
# counter_dict = {}
# for key in d1:
#     c = Counter(d1[key]).most_common(1)[0][0]
#     counter_dict[key] = c


In [629]:
# import pickle

# f = open("dict.pkl","wb")
# pickle.dump(counter_dict,f)
# f.close()

In [630]:
test[test['before'] == 'L'].head(50)

,Unnamed: 0,sentence_id,token_id,before,class
14067,14067,1115,10,L,LETTERS
37686,37686,2911,20,L,LETTERS
54459,54459,4205,5,L,LETTERS
57880,57880,4473,13,L,PLAIN
59725,59725,4611,18,L,PLAIN
59735,59735,4611,28,L,PLAIN
67517,67517,5188,6,L,PLAIN
78120,78120,5996,1,L,PLAIN
82804,82804,6352,7,L,PLAIN
88083,88083,6743,9,L,PLAIN


In [631]:
train[train['before'] == 'XXI'].head(50)

,sentence_id,token_id,class,before,after
55280,4446,2,CARDINAL,XXI,twenty one
758155,60409,9,CARDINAL,XXI,twenty one
1192034,94027,14,CARDINAL,XXI,twenty one
1298154,102191,16,ORDINAL,XXI,twenty first
1759608,137554,0,CARDINAL,XXI,twenty one
2371320,184269,17,CARDINAL,XXI,twenty one
2589041,200785,1,CARDINAL,XXI,twenty one
3012774,232765,4,CARDINAL,XXI,twenty one
3678386,283135,2,CARDINAL,XXI,twenty one
3747286,288339,3,CARDINAL,XXI,twenty one


In [632]:
# f = open("dict.pkl","wb")
# pickle.dump(counter_dict,f)
# f.close()

p1 = pickle.load(open('dict.pkl', 'rb'))

# p1['the']

In [633]:
# import os

# p = {} # scores is an empty dict already
# target = 'dict.pkl'
# if os.path.getsize(target) > 0:      
#     with open(target, "rb") as f:
#         unpickler = pickle.Unpickler(f)
#         # if file is not empty scores will be equal
#         # to the value unpickled
#         p = unpickler.load()

### Convert roman numerals to integers

In [634]:
def rom_to_int(string):

    table=[['M',1000],['CM',900],['D',500],['CD',400],['C',100],['XC',90],
           ['L',50],['XL',40],['X',10],['IX',9],['V',5],['IV',4],['I',1]]
    returnint=0
    for pair in table:


        continueyes=True

        while continueyes:
            if len(string)>=len(pair[0]):

                if string[0:len(pair[0])]==pair[0]:
                    returnint+=pair[1]
                    string=string[len(pair[0]):]

                else: continueyes=False
            else: continueyes=False

    return returnint

### Define classes for every type of word

In [2]:
## For classes:
p = inflect.engine()
def plain(x):
    if x == '-':
        return 'to'
    else:
        return x

def punct(x):
    return x

def cardinal(x):
    try:
        if re.match('.*[A-Za-z]+.*', x):
            return x
        x = re.sub(',', '', x, count = 10)

        if(re.match('.+\..*', x)):
            x = p.number_to_words(float(x))
        elif re.match('\..*', x): 
            x = p.number_to_words(float(x))
            x = x.replace('zero ', '', 1)
        else:
            x = p.number_to_words(int(x))
        x = x.replace('zero', 'o')    
        x = re.sub('-', ' ', x, count=10)
        x = re.sub(' and','',x, count = 10)
        x = re.sub(',','',x, count = 10)
        return x
    except:
        return x

def verbatim(x):
    return(x)


# def date(key):
#     dict_mon = {'jan': "January", "feb": "February", "mar ": "march", "apr": "april", "may": "may ","jun": "june", "jul": "july", "aug": "august","sep": "september",
#             "oct": "october","nov": "november","dec": "december", "january":"January", "february":"February", "march":"march","april":"april", "may": "may", 
#             "june":"june","july":"july", "august":"august", "september":"september", "october":"october", "november":"november", "december":"december"}


#     v =  key.split('-')
#     if len(v)==3:
#         if v[1].isdigit():
#             try:
#                 date = datetime.strptime(key , '%Y-%m-%d')
#                 text = 'the '+ p.ordinal(p.number_to_words(int(v[2]))).replace('-',' ')+' of '+datetime.date(date).strftime('%B')
#                 if int(v[0])>=2000 and int(v[0]) < 2010:
#                     text = text  + ' '+cardinal(v[0])
#                 else: 
#                     text = text + ' ' + cardinal(v[0][0:2]) + ' ' + cardinal(v[0][2:])
#             except:
#                 text = key
#             return text.lower()    
#     else:   
#         v = re.sub(r'[^\w]', ' ', key).split()
#         if v[0].isalpha():
#             try:
#                 if len(v)==3:
#                     text = dict_mon[v[0].lower()] + ' '+ p.ordinal(p.number_to_words(int(v[1]))).replace('-',' ')
#                     if int(v[2])>=2000 and int(v[2]) < 2010:
#                         text = text  + ' '+cardinal(v[2])
#                     else: 
#                         text = text + ' ' + cardinal(v[2][0:2]) + ' ' + cardinal(v[2][2:])   
#                 elif len(v)==2:

#                     if int(v[1])>=2000 and int(v[1]) < 2010:
#                         text = dict_mon[v[0].lower()]  + ' '+ cardinal(v[1])
#                     else: 
#                         if len(v[1]) <=2:
#                             text = dict_mon[v[0].lower()] + ' ' + cardinal(v[1])
#                         else:
#                             text = dict_mon[v[0].lower()] + ' ' + cardinal(v[1][0:2]) + ' ' + cardinal(v[1][2:])
#                 else: text = key
#             except: text = key
#             return text.lower()
#         else: 
#             key = re.sub(r'[^\w]', ' ', key)
#             v = key.split()
#             try:
#                 date = datetime.strptime(key , '%d %b %Y')
#                 text = 'the '+ p.ordinal(p.number_to_words(int(v[0]))).replace('-',' ')+' of '+ dict_mon[v[1].lower()]
#                 if int(v[2])>=2000 and int(v[2]) < 2010:
#                     text = text  + ' '+cardinal(v[2])
#                 else: 
#                     text = text + ' ' + cardinal(v[2][0:2]) + ' ' + cardinal(v[2][2:])
#             except:
#                 try:
#                     date = datetime.strptime(key , '%d %B %Y')
#                     text = 'the '+ p.ordinal(p.number_to_words(int(v[0]))).replace('-',' ')+' of '+ dict_mon[v[1].lower()]
#                     if int(v[2])>=2000 and int(v[2]) < 2010:
#                         text = text  + ' '+cardinal(v[2])
#                     else: 
#                         text = text + ' ' + cardinal(v[2][0:2]) + ' ' + cardinal(v[2][2:])
#                 except:
#                     try:
#                         date = datetime.strptime(key , '%d %m %Y')
#                         text = 'the '+ p.ordinal(p.number_to_words(int(v[0]))).replace('-',' ')+' of '+datetime.date(date).strftime('%B')
#                         if int(v[2])>=2000 and int(v[2]) < 2010:
#                             text = text  + ' '+cardinal(v[2])
#                         else: 
#                             text = text + ' ' + cardinal(v[2][0:2]) + ' ' + cardinal(v[2][2:])
#                     except:
#                         try:
#                             date = datetime.strptime(key , '%d %m %y')
#                             text = 'the '+ p.ordinal(p.number_to_words(int(v[0]))).replace('-',' ')+' of '+datetime.date(date).strftime('%B')
#                             v[2] = datetime.date(date).strftime('%Y')
#                             if int(v[2])>=2000 and int(v[2]) < 2010:
#                                 text = text  + ' '+cardinal(v[2])
#                             else: 
#                                 text = text + ' ' + cardinal(v[2][0:2]) + ' ' + cardinal(v[2][2:])
#                         except:text = key
#             return text.lower() 




def date(x):
    dict_mon = {'jan': "January", "feb": "February", "mar ": "march", "apr": "april", "may": "may ","jun": "june", "jul": "july", "aug": "august","sep": "september",
            "oct": "october","nov": "november","dec": "december", "january":"January", "february":"February", "march":"march","april":"april", "may": "may", 
            "june":"june","july":"july", "august":"august", "september":"september", "october":"october", "november":"november", "december":"december"}


    try:
        x = re.sub('\.','',x )
        if re.match('^[0-9]+$', x):
            return re.sub('-', ' ', p.number_to_words(int(x))) 
        
        elif re.match('.*[A-Za-z]+.*',x):
            y = re.sub('-','',x )
            y = re.sub(',','',x)
            y = y.split(' ')
            result_string = 'the '
            two_dig = [w for w in y if len(w) <= 2]
            if len(two_dig)>0:
                num = int(str(two_dig[0]))
                num = re.sub('-', ' ',  num2words(num, ordinal=True))
                result_string = result_string + num + ' of '
            month = [w for w in y if re.match('^[A-Za-z]+$', w)]
            if month[0].lower() in dict_mon:
                month1 = dict_mon[month[0].lower()]
                if result_string.split(' ')[1] == '':
                    result_string = result_string[3:]
                result_string = result_string  + month1
            else:    
                result_string = result_string  + str(month[0])
            y = [w for w in y if w not in month]
            year = [w for w in y if len(w) == 4]
            year1 = (str(year[0][0:2]))
            year2 = (str(year[0][2:4]))
            year1 = re.sub('-', ' ', p.number_to_words(int(year1)))
            year2 = re.sub('-', ' ', p.number_to_words(int(year2)))
            result_string = result_string + ' ' + year1 + ' ' + year2
            return result_string.strip()
        else:
            result_string = 'the '
            y = re.sub('-', ' ', x)
            y = y.split(' ')
            num = int(str(y[0]))
            num = re.sub('-', ' ',  num2words(num, ordinal=True))
            result_string = result_string + num + ' of '
            month = y[1]
            result_string = result_string  + str(month[0])
            year = [w for w in y if len(w) == 4]
            year1 = (str(year[0][0:2]))
            year2 = (str(year[0][2:4]))
            year1 = re.sub('-', ' ', p.number_to_words(int(year1)))
            year2 = re.sub('-', ' ', p.number_to_words(int(year2)))
            result_string = result_string + ' ' + year1 + ' ' + year2
            return result_string.strip()
    except:
        return x
    
def measure(x):
    try:

        x = re.sub(',', '', x, count=10)
        x = x.split(' ')

        #replacement = {'MB': 'megabyte', 'MW': 'megawatt','ft':'feet', 'km':'kilometers', 'mm': 'millimeters', 'ha':'hectares', '"':'inches', 'cm':'centimeters', '/day' : 'per day', 'nm' : 'nanometers', '/s':'per second', 'm2':'square meters', 'km2': 'square kilometers', 'percent': 'percent'}
        #Comprehensive list of all measures
        replacement = {'"': 'inches','inch':'inch', 'inches':'inches', "'": 'feet', 'km/s': 'kilometers per second','kilometers per second':'kilometers per second', 'AU': 'units', 'BAR': 'bars', 'CM': 'centimeters', 'mm': 'millimeters', 'FT': 'feet', 'G': 'grams', 
     'GAL': 'gallons', 'GB': 'gigabytes', 'GHZ': 'gigahertz', 'HA': 'hectares', 'HP': 'horsepower', 'HZ': 'hertz', 'KM':'kilometers', 'km3': 'cubic kilometers',
     'KA':'kilo amperes', 'KB': 'kilobytes', 'KG': 'kilograms', 'KHZ': 'kilohertz', 'KM²': 'square kilometers', 'KT': 'knots', 'KV': 'kilo volts', 'M': 'meters',
     'KM2': 'square kilometers','Kw':'kilowatts', 'KWH': 'kilo watt hours', 'LB': 'pounds', 'LBS': 'pounds', 'MA': 'mega amperes', 'MB': 'megabytes',
     'KW': 'kilowatts', 'MPH': 'miles per hour', 'MS': 'milliseconds', 'MV': 'milli volts', 'kJ':'kilojoules', 'km/h': 'kilometers per hour',  'V': 'volts', 
     'M2': 'square meters', 'M3': 'cubic meters', 'MW': 'megawatts', 'M²': 'square meters', 'M³': 'cubic meters', 'OZ': 'ounces',  'MHZ': 'megahertz', 'MI': 'miles',
     'MB/S': 'megabytes per second', 'MG': 'milligrams', 'ML': 'milliliters', 'YD': 'yards', 'au': 'units', 'bar': 'bars', 'cm': 'centimeters', 'ft': 'feet', 'g': 'grams', 
     'gal': 'gallons', 'gb': 'gigabytes', 'ghz': 'gigahertz', 'ha': 'hectares', 'hp': 'horsepower', 'hz': 'hertz', 'kWh': 'kilo watt hours', 'ka': 'kilo amperes', 'kb': 'kilobytes', 
     'kg': 'kilograms', 'khz': 'kilohertz', 'km': 'kilometers', 'km2': 'square kilometers', 'km²': 'square kilometers', 'kt': 'knots','kv': 'kilo volts', 'kw': 'kilowatts', 
     'lb': 'pounds', 'lbs': 'pounds', 'm': 'meters', 'm2': 'square meters','m3': 'cubic meters', 'ma': 'mega amperes', 'mb': 'megabytes', 'mb/s': 'megabytes per second', 
     'mg': 'milligrams', 'mhz': 'megahertz', 'mi': 'miles', 'ml': 'milliliters', 'mph': 'miles per hour','ms': 'milliseconds', 'mv': 'milli volts', 'mw': 'megawatts', 'm²': 'square meters',
     'm³': 'cubic meters', 'oz': 'ounces', 'v': 'volts', 'yd': 'yards', 'µg': 'micrograms', 'ΜG': 'micrograms', 'kg/m3': 'kilograms per meter cube', 'nautical miles':'nautical miles', 'metres' : 'meters',
     'hectares':'hectares','hectare':'hectare', 'acre':'acres', 'acres':'acres', 'meters':'meters', 'centimeters':'centimeters', 'kilometers':'kilometers', 'years':'years', 'feet':'feet', 'yrs':'years', 'yr': 'year', 'year': 'year',
     'hours':'hours', 'hrs':'hours', 'hour':'hour', 'hr':'hr', 'miles':'miles', 'mile': 'mile', 'week':'week', 'weeks':'weeks', 'kilometres':'kilometres', 'centimetres': 'centimeters','square':'square','yard':'yard', 'yards':'yards', 'millimeters':'millimeters','millimeter':'millimeter', 'second':'second', 
     'seconds':'seconds', 'minute':'minute', 'minutes':'minutes', 'nautical':'nautical', 'month':'month', 'months':'months', 'ton':'ton', 'tons':'tons', 'grams':'grams', 'gram':'gram', '/sq mi':'per square miles', 'kms':'kilometers', 'million':'million', 'thousand':'thousand'}
        result_string = ''
        if re.match('.*/km²', x[0]):
            x = re.sub('/km²','',x[0])
            x = cardinal(x)
            result_string = result_string + x + ' per square kilometers' 
            return(result_string)
        if re.match('.*%$',x[0]):
            x = re.sub('%','',x[0])
            x = cardinal(x)
            result_string = result_string + x + ' percent' 
        elif re.match('.*\"$',x[0]):
            x = re.sub('\"','',x[0])
            x = cardinal(x)
            result_string = result_string + x + ' inches' 
        elif re.match('.*/sq mi$', str(x[0] + ' '+x[1])):
            x = re.sub('/sq mi','',str(x[0] + ' '+x[1]))
            x = cardinal(x)
            result_string = result_string + x + ' ' + replacement['/sq mi']  
 
        elif len(x)<2:
            return x
        elif re.match('.*ft$', x[0]):
            x = re.sub('ft','',x[0])
            x = cardinal(x)
            result_string = result_string + x1 + ' ' + replacement['ft']
          
        elif x[1] in replacement:
            x1 = cardinal(x[0])

            result_string = result_string + x1 + ' ' + replacement[x[1]] 
            if len(x) == 3:
                result_string = result_string + ' ' + replacement[x[2]]
        else:
            result_string = x
            result_string = re.sub(',','',result_string)
        return(result_string)
    except:
        return x

def letters(x):
    try:
        x = re.sub('[^a-zA-Z]', '', x)
        x = x.lower()
        result_string = ''
        for i in range(len(x)):
            result_string = result_string + x[i] + ' '
        return(result_string.strip())  
    except:
        print(x)
        return x
    
    
def decimal(x):
    try:
        x = re.sub(',', '',x, count=10)
        x = x.split(' ')
        if len(x) == 1:
            result_string = cardinal(x[0])
        else:
            result_string = cardinal(x[0]) + ' ' + x[1]
        return result_string   
    except:
        return x

def ordinal(x):
    try:
        result_string = ''
        x = x.replace(',', '')
        x = x.replace('[\.]$', '')
        if re.match('^[0-9]+$',x):
            x = num2words(int(x), ordinal=True)
            return(x.replace('-', ' '))
        if re.match('.*V.*',x) or re.match('.*X.*',x) or re.match('.*I.*', x) or re.match('.*L.*', x) or re.match('.*D.*',x):
            if re.match('.*th',x) or re.match('.*st',x) or re.match('.*nd', x) or re.match('.*rd',x):
                x = x[0:len(x)-2]
                x = rom_to_int(x)
                result_string = re.sub('-', ' ',  num2words(x, ordinal=True))
            else:
                x = rom_to_int(x)
                result_string = 'the '+ re.sub('-', ' ',  num2words(x, ordinal=True))
        else:
            x = x[0:len(x)-2]
            result_string = re.sub('-', ' ',  num2words(float(x), ordinal=True))
        return(result_string)  
    except:
        return x

def electronic(x):
    try:
        replacement = {'.' : 'dot', ':' : 'colon', '/':'slash', '-' : 'dash', '#' : 'hash tag', }
        result_string = ''
        if re.match('.*[A-Za-z].*', x):
            for char in x:
                if re.match('[A-Za-z]', char):
                    result_string = result_string + letters(char) + ' '
                elif char in replacement:
                    result_string = result_string + replacement[char] + ' '
                elif re.match('[0-9]', char):
                    if char == 0:
                        result_string = result_string + 'o '
                    else:
                        number = cardinal(char)
                        for n in number:
                            result_string = result_string + n + ' ' 
            return result_string.strip()                
        else:
            return(x)
    except:    
        return(x)

def address(x):
    try:
        x = re.sub('[^0-9a-zA-Z]+', '', x)
        result_string = ''
        for i in range(0,len(x)):
            if re.match('[A-Z]|[a-z]',x[i]):
                result_string = result_string + plain(x[i]).lower() + ' '
            else:
                result_string = result_string + cardinal(x[i]) + ' '
                
        return(result_string.strip())        
    except:    
        return(x)

def telephone(x):
    try:
        result_string = ''
        for i in range(0,len(x)):
            if re.match('[0-9]+', x[i]):
                result_string = result_string + cardinal(x[i]) + ' '
            else:
                result_string = result_string + 'sil '
        return result_string.strip()    
    except:    
        return(x)

# def time(x):
#     try:
#         if re.match('[0-9]*:[0-9]',key): 
#             v = key.split()
#             if len(v) == 1:
#                 v[0] = v[0].split(':')
#                 try:
#                     text = ' '.join([cardinal(i) for i in v[0]])
#                 except:
#                     print("5: " + key)
#                     text = key
#             elif len(v) == 2:
#                 v[0] = v[0].split(':')
#                 v[1] = v[1].replace('.','')
#                 v[1] = " ".join(v[1])
#                 text = ' '.join([cardinal(i) for i in v[0]]) + ' ' + v[1]
#             else: 
#                 text = key
#                 print("5.2: " + key)
#             return(text.lower())
#     except:
#         return(x)
# #         print(key + ':'+text)
def time(x):
    try:
        x = re.sub('\.','',x)
        if len(x.split(':')) == 2:
            x = re.sub(':',' ',x)
            x = re.sub('([^0-9])', r' \1 ', x)
            x = re.sub('\s{2,}', ' ', x)
            time_list = x.split(' ')
            t_list = [i for i in time_list if i != ""] 
            for i,v in enumerate(t_list):
                if v == '00':
                    t_list[i] = ''
                elif v.isdigit():
                    t_list[i] = num2words(int(v))
                else:
                    t_list[i] = v.lower()
            t = " ".join(t_list)
        elif len(x.split(':')) == 3:
            x_list = x.split(':')
            time_list = [num2words(int(num)) for num in x_list]
            time_units = []
            if int(x_list[0]) != 1:
                time_units.append('hours')
            else:
                time_units.append('hour')
            if int(x_list[1]) != 1:
                time_units.append('minutes')
            else:
                time_units.append('minute')
            if int(x_list[2]) != 1:
                time_units.append('seconds')
            else:
                time_units.append('second')
            t_list = [time_list[0],time_units[0],time_list[1],time_units[1],time_list[2],time_units[2]]
            t = " ".join(t_list)
        else:
            x = re.sub('([^0-9])', r' \1 ', x)
            x = re.sub('\s{2,}', ' ', x)
            time_list = x.split(' ')
            t_list = [i for i in time_list if i != ""] 
            for i,v in enumerate(t_list):
                if v == '00':
                    t_list[i] = ''
                elif v.isdigit():
                    t_list[i] = num2words(int(v))
                else:
                    t_list[i] = v.lower()
            t = " ".join(t_list)       

        time = re.sub(',',"",t)
        time_final = re.sub('-'," ",time)
        return(time_final)
    except:
        return(x)


def money(x):
    try:
        if re.match('^\$', x):
            x = x.replace('$','')
            x = x.strip()
            if len(x.split(' ')) == 1:
                if re.match('.*M$',x) or re.match('.*m$',x):
                    x = x.replace('M', '')
                    x = x.replace('m', '')
                    text = cardinal(x)
                    x = text + ' million dollars'
                elif re.match('.*b$', x) or re.match('.*B$', x) or re.match('.*bn$', x):
                    x = x.replace('B', '')
                    x = x.replace('bn', '')
                    x = x.replace('b', '')
                    text = cardinal(x)
                    x = text + ' billion dollars'
                else:
                    text = cardinal(x)
                    x = text + ' dollars'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'million':
                    x = text + ' million dollars'
                elif x.split(' ')[1].lower() == 'billion':
                    x = text + ' billion dollars'
                return x.lower()
                
                
                
        elif re.match('^US\$', x):
            x = x.replace('US$','')
            x = x.strip()
            if len(x.split(' ')) == 1:
                if re.match('.*M$',x) or re.match('.*m$',x):
                    x = x.replace('M', '')
                    x = x.replace('m', '')
                    text = cardinal(x)
                    x = text + ' million dollars'
                elif re.match('.*b$', x) or re.match('.*B$', x) or re.match('.*bn$', x):
                    x = x.replace('bn', '')
                    x = x.replace('b', '')
                    x = x.replace('B', '')
                    text = cardinal(x)
                    x = text + ' billion dollars'
                else:
                    text = cardinal(x)
                    x = text + ' dollars'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'million':
                    x = text + ' million dollars'
                elif x.split(' ')[1].lower() == 'billion':
                    x = text + ' billion dollars'
                return x.lower()
                
        elif re.match('^USD', x):
            x = x.replace('USD','')
            x = x.strip()
            if len(x.split(' ')) == 1:
                if re.match('.*M$',x) or re.match('.*m$',x):
                    x = x.replace('M', '')
                    x = x.replace('m', '')
                    text = cardinal(x)
                    x = text + ' million dollars'
                elif re.match('.*b$', x) or re.match('.*B$', x) or re.match('.*bn$', x):
                    x = x.replace('bn', '')
                    x = x.replace('b', '')
                    x = x.replace('B', '')
                    text = cardinal(x)
                    x = text + ' billion dollars'
                else:
                    text = cardinal(x)
                    x = text + ' dollars'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'million':
                    x = text + ' million dollars'
                elif x.split(' ')[1].lower() == 'billion':
                    x = text + ' billion dollars'
                return x.lower()
            
        elif re.match('^£', x):
            x = x.replace('£','')
            x = x.strip()
            if len(x.split(' ')) == 1:
                if re.match('.*M$',x) or re.match('.*m$',x):
                    x = x.replace('M', '')
                    x = x.replace('m', '')
                    text = cardinal(x)
                    x = text + ' million pounds'
                elif re.match('.*b$', x) or re.match('.*B$', x) or re.match('.*bn$', x):
                    x = x.replace('bn', '')

                    x = x.replace('b', '')
                    x = x.replace('B', '')
                    text = cardinal(x)
                    x = text + ' billion pounds'
                else:
                    text = cardinal(x)
                    x = text + ' pounds'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'million':
                    x = text + ' million pounds'
                elif x.split(' ')[1].lower() == 'billion':
                    x = text + ' billion pounds'
                return x.lower()
            
        elif re.match('^€', x):
            x = x.replace('€','')
            x = x.strip()
            if len(x.split(' ')) == 1:
                if re.match('.*M$',x) or re.match('.*m$',x):
                    x = x.replace('M', '')
                    x = x.replace('m', '')
                    text = cardinal(x)
                    x = text + ' million euros'
                elif re.match('.*b$', x) or re.match('.*B$', x) or re.match('.*bn$', x):
                    x = x.replace('bn', '')

                    x = x.replace('B', '')
                    x = x.replace('b', '')
                    text = cardinal(x)
                    x = text + ' billion euros'
                else:
                    text = cardinal(x)
                    x = text + ' euros'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'million':
                    x = text + ' million euros'
                elif x.split(' ')[1].lower() == 'billion':
                    x = text + ' billion euros'
                return x.lower()
        
        elif re.match('^¥', x):
            x = x.replace('¥','')
            x = x.strip()
            if len(x.split(' ')) == 1:
                if re.match('.*M$',x) or re.match('.*m$',x):
                    x = x.replace('M', '')
                    x = x.replace('m', '')
                    text = cardinal(x)
                    x = text + ' million yen'
                elif re.match('.*b$', x) or re.match('.*B$', x) or re.match('.*bn$', x):
                    x = x.replace('bn', '')

                    x = x.replace('B', '')
                    x = x.replace('b', '')
                    text = cardinal(x)
                    x = text + ' billion yen'
                else:
                    text = cardinal(x)
                    x = text + ' yen'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'million':
                    x = text + ' million yen'
                elif x.split(' ')[1].lower() == 'billion':
                    x = text + ' billion yen'
                return x.lower()   
            
        elif re.match('^Rs', x):
            x = x.replace('Rs','')
            x = x.strip()
            if len(x.split(' ')) == 1:
                text = cardinal(x)
                x = text + ' rupees'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'crores':
                    x = text + ' crore rupees'
                return x.lower()    
                        
            
        elif x.endswith('meters') or x.endswith('meters') or x.endswith('centimetres') or x.endswith('acres') or x.endswith('yards') or x.endswith('nautical miles') or x.endswith('square miles') or x.endswith('hectares') or x.endswith('metres') or x.endswith('hours') or x.endswith('months') or x.endswith('month') or x.endswith('miles') or x.endswith('per hour') or x.endswith('years') or x.endswith('seconds') or x.endswith('weeks') or x.endswith('tons'):
            return measure(x)
        elif x.endswith('EDT') or x.endswith('PDT'):
            return time(x)
    except:    
        return(x)

def fraction(x):
    try:
        if re.match('.*,.*',x):
            return cardinal(x)
        y = x.split('/')
        result_string = ''
        y[0] = cardinal(y[0])
        y[1] = ordinal(y[1])
        if y[1] == 4:
            result_string = y[0] + ' quarters'
        else:    
            result_string = y[0] + ' ' + y[1] + 's'
        return(result_string)
    except:    
        return(x)
    
    
def digit(x): 
    try:
        x = re.sub('[^0-9]', '',x)
        result_string = ''
        for i in x:
            result_string = result_string + cardinal(i) + ' '
        result_string = result_string.strip()
        return result_string
    except:
        return(x)

NameError: name 'inflect' is not defined

In [636]:
x = '3.3/km²'
measure(x)
x = '0'
cardinal(x)

'o'

### Generate Predictions

In [637]:
after = []
count_train = 0
count_pkl = 0
count_mine = 0
for i in range(test.shape[0]):

    class_name = test.iloc[i,4]
    if class_name == 'ORDINAL':
        after.append(ordinal(test.iloc[i,3]))    
    elif class_name == 'CARDINAL' and test.iloc[i,3] == '0':
        after.append('zero')    
    elif class_name == 'CARDINAL' and len(test.iloc[i,3])>5:
        after.append(cardinal(test.iloc[i,3]))
    
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'I':
        after.append('one')
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'II':
        after.append('two')
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'III':
        after.append('three')          
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'IV':
        after.append('four')        
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'V':
        after.append('five')
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'VI':
        after.append('six')   
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'VII':
        after.append('seven') 
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'VIII':
        after.append('eight')          
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'IX':
        after.append('nine')
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'X':
        after.append('ten')          
    elif class_name == 'CARDINAL' and test.iloc[i,3] == 'XI':
        after.append('eleven')          
             
        
    elif test.iloc[i,3] == '-':                          
        if class_name == 'PLAIN':
            after.append(plain(test.iloc[i,3]))
        else:
            after.append(verbatim(test.iloc[i,3]))
            
    elif class_name == 'PUNCT':                          # Punctuations
        after.append(punct(test.iloc[i,3]))        

    elif test.iloc[i,3] in train_dict.keys():            # train_dict
        if train_dict[test.iloc[i,3]] == 'sil':
            after.append(test.iloc[i,3])
        else:    
            after.append(train_dict[test.iloc[i,3]])
        count_train = count_train+1
    elif test.iloc[i,3] in test_dict.keys():             #test_dict
        if test_dict[test.iloc[i,3]] == 'sil':
            after.append(test.iloc[i,3])
        else:    
            after.append(test_dict[test.iloc[i,3]])            
#     elif test.iloc[i,3] in extra_dict.keys():          #extra dict
#         after.append(extra_dict[test.iloc[i,3]])  
    elif test.iloc[i,3] in p1.keys():                    #pickle
        count_pkl = count_pkl+1
        if p1[test.iloc[i,3]] == '<self>':
            after.append(test.iloc[i,3])
        elif p1[test.iloc[i,3]] == 'sil':
            after.append(test.iloc[i,3])
        else:
            after.append(p1[test.iloc[i,3]])
          
    else:    
        count_mine = count_mine+1
        class_name = test.iloc[i,4]
        if class_name == 'PLAIN':
            after.append(plain(test.iloc[i,3]))
        elif class_name == 'PUNCT':
            after.append(punct(test.iloc[i,3]))
        elif class_name == 'LETTERS':                        # Letters
            after.append(letters(test.iloc[i,3])) 


        elif class_name == 'VERBATIM':
            after.append(verbatim(test.iloc[i,3])) 
        elif class_name == 'DATE':
            after.append(date(test.iloc[i,3]))
        elif class_name == 'MEASURE':
            after.append(measure(test.iloc[i,3]))

        elif class_name == 'DECIMAL':
            after.append(decimal(test.iloc[i,3]))
        elif class_name  == 'CARDINAL':                      # Cardinal
            after.append(cardinal(test.iloc[i,3]))         
        elif class_name == 'ELECTRONIC':
            after.append(electronic(test.iloc[i,3]))
        elif class_name == 'ADDRESS':
            after.append(address(test.iloc[i,3]))
        elif class_name == 'DIGIT':
            after.append(digit(test.iloc[i,3]))
        elif class_name == 'MONEY':
            after.append(money(test.iloc[i,3]))
        elif class_name == 'TIME':
            after.append(time(test.iloc[i,3]))
        elif class_name == 'TELEPHONE':
            after.append(telephone(test.iloc[i,3]))
        elif class_name == 'FRACTION':
            after.append(fraction(test.iloc[i,3]))
#     print(test.iloc[i,3] + after[i])

       

In [638]:
print(count_train)
print(count_pkl)
print(count_mine)

715019
29492
12073


### Sanity Check!!

In [639]:
train[train['class'] == 'CARDINAL'].head(200)

,sentence_id,token_id,class,before,after
95,6,3,CARDINAL,91,ninety one
116,9,3,CARDINAL,3,three
143,10,8,CARDINAL,35,thirty five
161,14,4,CARDINAL,2,two
280,24,4,CARDINAL,3,three
426,33,10,CARDINAL,4,four
450,34,20,CARDINAL,935,nine hundred thirty five
477,38,4,CARDINAL,1065520,one million sixty five thousand five hundred t...
717,54,4,CARDINAL,9,nine
783,59,4,CARDINAL,3,three


In [640]:
test[test['class'] == 'PLAIN'].head(1000)


,Unnamed: 0,sentence_id,token_id,before,class
0,0,0,0,Last,PLAIN
1,1,0,1,modified,PLAIN
4,4,1,0,There's,PLAIN
5,5,1,1,More,PLAIN
6,6,1,2,to,PLAIN
7,7,1,3,Clear,PLAIN
8,8,1,4,Channel,PLAIN
9,9,1,5,Than,PLAIN
11,11,1,7,The,PLAIN
12,12,1,8,Larry,PLAIN


In [641]:
test['after'] = after
test.head()
test[test['class'] == 'CARDINAL'].head(1000)

,Unnamed: 0,sentence_id,token_id,before,class,after
127,127,11,4,17,CARDINAL,seventeen
264,264,25,12,5,CARDINAL,five
343,343,31,7,269,CARDINAL,two hundred sixty nine
346,346,31,10,2,CARDINAL,two
403,403,36,7,9,CARDINAL,nine
420,420,38,1,9,CARDINAL,nine
435,435,39,7,582,CARDINAL,five hundred eighty two
532,532,49,4,2,CARDINAL,two
535,535,49,7,0,CARDINAL,zero
537,537,49,9,13,CARDINAL,thirteen


In [642]:
test[test['before'] == '0']

,Unnamed: 0,sentence_id,token_id,before,class,after
535,535,49,7,0,CARDINAL,zero
6893,6893,564,6,0,CARDINAL,zero
9763,9763,781,1,0,CARDINAL,zero
22813,22813,1771,5,0,CARDINAL,zero
31611,31611,2451,4,0,CARDINAL,zero
42352,42352,3274,1,0,CARDINAL,zero
111562,111562,8515,11,0,CARDINAL,zero
113570,113570,8671,10,0,CARDINAL,zero
130440,130440,9914,4,0,CARDINAL,zero
131760,131760,10011,4,0,CARDINAL,zero


In [643]:
test['id'] = test.sentence_id.astype(str) + '_' + test.token_id.astype(str)

print(test[['id', 'after']])
test[['id', 'after']].to_csv('output17.csv', index=False)
# add replacement for roman numbers .38
# output 16 - .39 #kerem rank 23 after replacing sil
# output 17: change 0, cardinal ka big values

              id                                     after
0            0_0                                      Last
1            0_1                                  modified
2            0_2  the thirty first of march twenty sixteen
3            0_3                                         .
4            1_0                                   There's
5            1_1                                      More
6            1_2                                        to
7            1_3                                     Clear
8            1_4                                   Channel
9            1_5                                      Than
10           1_6                                         '
11           1_7                                       The
12           1_8                                     Larry
13           1_9                                      King
14          1_10                                      Show
15          1_11                                        

## XGBoost to predict class of word

In [10]:
# -*- coding: utf-8 -*-
# @Time    : 2017/9/30 8:53
# @Author  : LiYun
# @File    : main_v2.py
'''description:
this method is a simple extention of BingQing Wei's XGboost With Context Label Data (ACC: 99.637%)
it's accuracy is 99.81% when 10% of the training data is used as validtion data
and finally, the whole data is used for training
'''
import os
import gc
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [11]:
def get_classify_train_data(np_file,csv_file):
    if os.path.exists(np_file) == True:
       temp = np.load(np_file)
       return temp['x_train'],temp['y_train'],temp['label']
    else:
        num_features = 9 #每个 word 取前 5 后 4 个字符来编码
        train=pd.read_csv(csv_file)
        tmp=pd.factorize(train['class'])
        y_train,label=tmp[0].astype(np.int8),tmp[1].values
        num_train=len(y_train)
        train['before']=train['before'].astype(np.str)
        x_train=np.zeros([num_train,num_features],np.int8)
        feature=np.zeros([num_train,7],np.int8)# 人工提取的特征
        list1=('a','e','i','o','u')# 元音
        list2=('+','-','*','//','%')# 数学运算符
        for word,row in zip(train['before'].values,range(num_train)):
            if(len(word)>=num_features):
                for c,col in zip(word[:5],range(5)):
                    x_train[row,col]=ord(c)
                for c,col in zip(word[-4:],range(5,9)):
                    x_train[row,col]=ord(c)
            else:
                for c,col in zip(word,range(num_features)):
                    x_train[row,col]=ord(c)
            feature[row, 3] =len(word) # 统计字符串的长度
            dotflag=0
            for c in word:
                if c.isdigit():feature[row,0]+=1# 统计数字的个数
                if c.isupper():feature[row,1]+=1# 统计大写字母的个数
                if c.isalnum()!=True:feature[row,2]+=1# 统计非字母和数字的个数
                if c in list1:feature[row,4]+=1# 统计元音的个数
                if c=='.': dotflag=1
                elif dotflag==1:#  . 后面跟字母置 1 ，数字置 2，其他置 3
                    dotflag = 0
                    if c.isdigit():feature[row,5]+=10
                    elif c.isalpha():feature[row,5]+=100
                    else:feature[row,5]+=1000
                if c in list2:feature[row,6]+=1# 统计数学运算符的个数

        # 掐头去尾，结合上文 2 单词，下文 1 个单词
        num_train-=3
        y_train=y_train[2:-1]
        x_train=np.concatenate((x_train[:-3],x_train[1:-2],x_train[2:-1],x_train[3:],feature[2:-1]),axis=1)
        np.savez(np_file,x_train=x_train, y_train=y_train, label=label)
        return x_train, y_train, label

def get_classify_test_data(np_file,csv_file):
    test=pd.read_csv(csv_file)
    if os.path.exists(np_file) == True:
       temp = np.load(np_file)
       x_test=temp['x_test']
    else:
        num_features = 9 #每个 word 取前 5 后 4 个字符来编码
        human_feature=7 #人工提取7个特征
        num_test=len(test)
        test['before']=test['before'].astype(np.str)
        x_test=np.zeros([num_test,num_features],np.int8)
        feature=np.zeros([num_test,human_feature],np.int8)# 人工提取的特征
        list1=('a','e','i','o','u')# 元音
        list2=('+','-','*','//','%')# 数学运算符
        for word,row in zip(test['before'].values,range(num_test)):
            if(len(word)>=num_features):
                for c,col in zip(word[:5],range(5)):
                    x_test[row,col]=ord(c)
                for c,col in zip(word[-4:],range(5,9)):
                    x_test[row,col]=ord(c)
            else:
                for c,col in zip(word,range(num_features)):
                    x_test[row,col]=ord(c)
            feature[row, 3] =len(word) # 统计字符串的长度
            dotflag=0
            for c in word:
                if c.isdigit():feature[row,0]+=1# 统计数字的个数
                if c.isupper():feature[row,1]+=1# 统计大写字母的个数
                if c.isalnum()!=True:feature[row,2]+=1# 统计非字母和数字的个数
                if c in list1:feature[row,4]+=1# 统计元音的个数
                if c=='.': dotflag=1
                elif dotflag==1:#  . 后面跟字母置 1 ，数字置 2，其他置 3
                    dotflag = 0
                    if c.isdigit():feature[row,5]+=10
                    elif c.isalpha():feature[row,5]+=100
                    else:feature[row,5]+=1000
                if c in list2:feature[row,6]+=1# 统计数学运算符的个数

        # 开头补上2个单词,结尾补上1个单词，结合上文 2 单词，下文 1 个单词
        x_test = np.concatenate((np.zeros([2,num_features],np.int8),x_test,np.zeros([1,num_features],np.int8)),axis=0)
        feature = np.concatenate((np.zeros([2,human_feature],np.int8),feature,np.zeros([1,human_feature],np.int8)),axis=0)
        x_test=np.concatenate((x_test[:-3],x_test[1:-2],x_test[2:-1],x_test[3:],feature[2:-1]),axis=1)
        np.savez(np_file,x_test=x_test)
    return test, x_test

In [12]:
if __name__=='__main__':
    prehead=''
    train_data_csv='en_train.csv'
    classify_train_file='classify_train.npz'
    xgb_model='xgb_model.dat'
    test_data_csv='en_test_2.csv'
    classify_test_file='classify_test.npz'
    xgb_model2='xgb_model2.dat'
    classify_test_file2='classify_test2.npz'

    # 训练模型
    x_train,y_train,label=get_classify_train_data(prehead+classify_train_file,prehead+train_data_csv)
    print(x_train.shape)
    dtrain = xgb.DMatrix(x_train, label=y_train)
    watchlist = [(dtrain, 'train')]
    param = {
        'eta': 0.3,
        'max_depth':10,
        'objective':'multi:softmax',
        'num_class':len(label),
        'eval_metric':'merror',
        'subsample': 1,
        'colsample_bytree': 1,
        'silent':1,
        'seed':0,
    }
    num_boost_rounds=12
    model = xgb.train(param, dtrain, num_boost_rounds, watchlist,verbose_eval=1)
    print('save model ',xgb_model2)
    pickle.dump(model,open(xgb_model2,'wb'))# 保存模型
    del x_train,y_train
    gc.collect()

    # 预测 test 上的 class
    model = pickle.load(open(xgb_model2, "rb"))
    test,x_test=get_classify_test_data(prehead+classify_test_file,prehead+test_data_csv)
    print(x_test.shape)
    dtest = xgb.DMatrix(x_test)
    pred = model.predict(dtest)
    pred = [label[int(x)] for x in pred]
    test['class']=pred
    test.to_csv(os.path.join(prehead, 'test_pred_class_new.csv'))

(9918438, 43)
[0]	train-merror:0.006201
[1]	train-merror:0.005233
[2]	train-merror:0.004759
[3]	train-merror:0.004471
[4]	train-merror:0.004253
[5]	train-merror:0.004053
[6]	train-merror:0.003818
[7]	train-merror:0.003616
[8]	train-merror:0.003515
[9]	train-merror:0.003356
[10]	train-merror:0.003241
[11]	train-merror:0.003142
save model  xgb_model2.dat
(956046, 43)


In [13]:
len(pred)

956046

In [14]:
len(test)

956046

In [15]:
len(x_test)

956046

In [62]:
test_pred= pd.read_csv('test_pred_class.csv')
test = pd.read_csv('en_test.csv')
test_pred.head()

,Unnamed: 0,sentence_id,token_id,before,class
0,0,0,0,Another,PLAIN
1,1,0,1,religious,PLAIN
2,2,0,2,family,PLAIN
3,3,0,3,is,PLAIN
4,4,0,4,of,PLAIN


In [63]:
test.head()

,sentence_id,token_id,before
0,0,0,Another
1,0,1,religious
2,0,2,family
3,0,3,is
4,0,4,of


In [64]:
test_pred['before'] = test['before']

In [65]:
test_pred.head()

,Unnamed: 0,sentence_id,token_id,before,class
0,0,0,0,Another,PLAIN
1,1,0,1,religious,PLAIN
2,2,0,2,family,PLAIN
3,3,0,3,is,PLAIN
4,4,0,4,of,PLAIN


In [66]:
test_pred.to_csv('test_pred_class.csv')

In [ ]:
# print(test.head())
# test['class'].unique()

# sub.to_csv(base_path+'test_sub_xgb.csv', index=False)
# sub_train.to_csv(base_path+'train_sub_xgb_new.csv')

# X = train[['class', 'before']]
# X['class'] = X['class'].astype('category')
# X['before'] = [''.join(str(ord(c)) for c in s) for s in X['before']]
# y = train['after'].values
# y = y.map(ord)

# test['class'] = test['class'].astype('category')
# test['before'] = test['before'].map(ord)

